In [0]:
%run ./Containers

In [0]:
# Definindo configurações dos Containers
SRC_STORAGE_ACCOUNT = dbutils.secrets.get(scope="secrets-kv", key="storage-account")
SRC_SAS_TOKEN = dbutils.secrets.get(scope="secrets-kv", key="blob-sas-access")

containers_config = {
    "BALANCE": {
        "container": dbutils.secrets.get(scope="secrets-kv", key="src-balance-container"),
        "account": SRC_STORAGE_ACCOUNT,
        "sas": SRC_SAS_TOKEN
    },
    "CNPJ": {
        "container": dbutils.secrets.get(scope="secrets-kv", key="src-cnpj-container"),
        "account": SRC_STORAGE_ACCOUNT,
        "sas": SRC_SAS_TOKEN
    }
}
# Dicionário vazio para guardar os objetos instanciados
containers_registry = {}

In [0]:
# Instanciando Objetos
for container, config in containers_config.items():
    # Instancia o objeto para cada container
    obj = AzureContainer(
        storage_account=config["account"],
        container_name=config["container"],
        sas_token=config["sas"]
    )
    
    # Guarda o objeto no dicionário usando o apelido como chave
    containers_registry[container] = obj

    print(f"✅ Container '{container}' configurado com sucesso.")

In [0]:
# Lista para armazenar dicionários com os metadados
lista_arquivos_processada = []

for nome, obj in containers_registry.items():
    # Obtém a lista de arquivos do container
    arquivos = obj.get_files_list()
    
    for arq in arquivos:
        lista_arquivos_processada.append({
            "path": arq.path,
            "name": arq.name,
            "size": arq.size,
            "modificationTime": arq.modificationTime,
            "container_name": nome
        })

    print(f"✅ Metadados coletados de: {nome.upper()}")

# Criando o DataFrame com os arquivos
df_inventario = spark.createDataFrame(lista_arquivos_processada)

# Convertendo modificationTime para um formato mais amigável
df_inventario = df_inventario.withColumn(
    "modificationTime", 
    (F.col("modificationTime") / 1000).cast("timestamp")
)

display(df_inventario)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS bronze_1.bronze_control_table (
    container_name STRING,
    table_name STRING,
    input_file_name STRING,
    last_ingestion_timestamp TIMESTAMP
)
USING DELTA;